Descriptive Analysis  Question from schema_1

In [1]:
!pip install pandas

In [2]:
import pandas as pd

In [6]:
df1 = pd.read_excel(
    r"C:\Users\varle\Downloads\final clean data for py_hac\cleaned_schema_1_data_Final (1).xlsx"
)

df1.head()

,month,fsa,probable,vulnerable,is_most_recent,fever_chills_shakes,cough,shortness_of_breath,any_medical_conditions,travel_outside_canada,contact_with_illness,age_1,age_binary
0,April,L4C,0,0,1,0,0,0,0,0,0,<65,0
1,April,P1H,1,1,1,0,0,0,1,0,1,>65,1
2,April,N2J,0,1,1,0,0,0,0,0,0,>65,1
3,April,N1G,0,0,1,0,0,0,0,0,0,<65,0
4,April,L3M,0,0,1,0,0,0,0,0,0,<65,0


Question 1:Which variables have the highest data quality risk, and how might that bias insights?




In [8]:
import pandas as pd

df1 = pd.read_excel(
    r"C:\Users\varle\Downloads\final clean data for py_hac\cleaned_schema_1_data_Final (1).xlsx"
)

data_quality = pd.DataFrame({
    "missing_%": df1.isna().mean() * 100,
    "unique_values": df1.nunique(),
    "dtype": df1.dtypes
}).sort_values(by="missing_%", ascending=False)

data_quality

,missing_%,unique_values,dtype
month,0.0,1,object
fsa,0.0,516,object
probable,0.0,2,int64
vulnerable,0.0,2,int64
is_most_recent,0.0,1,int64
fever_chills_shakes,0.0,2,int64
cough,0.0,2,int64
shortness_of_breath,0.0,2,int64
any_medical_conditions,0.0,2,int64
travel_outside_canada,0.0,2,int64


Insight 
--All variables have 0% missing values, indicating strong data completeness.
However, the fsa column has very high cardinality (516 unique values), which can lead to sparse group-level statistics and unstable geographic insights.
Additionally, variables like month and is_most_recent show no variability, providing no analytical value and potentially introducing noise if retained.

Why this question matters

-Missing values, inconsistent formats, or extreme outliers can silently distort conclusions.
Understanding where data quality breaks helps you:

-Avoid misleading averages

-Decide between imputation vs. exclusion

-Justify preprocessing choices scientifically

Technical Insight

 what We evaluate:

-% missing values

-Data types

-Cardinality (unique values)

Question 2:Which numerical features show abnormal distributions or outliers, and what do they imply?


In [12]:
import numpy as np
import pandas as pd

num_cols = df1.select_dtypes(include=np.number).columns

outlier_summary = {}

for col in num_cols:
    Q1 = df1[col].quantile(0.25)
    Q3 = df1[col].quantile(0.75)
    IQR = Q3 - Q1
    
    outliers = df1[
        (df1[col] < Q1 - 1.5 * IQR) |
        (df1[col] > Q3 + 1.5 * IQR)
    ]
    
    outlier_summary[col] = len(outliers)

pd.Series(outlier_summary).sort_values(ascending=False)

any_medical_conditions    40574
age_binary                33903
probable                  11563
travel_outside_canada      9378
cough                      8818
contact_with_illness       6196
shortness_of_breath        5342
fever_chills_shakes        3303
vulnerable                    0
is_most_recent                0
dtype: int64

In [ ]:
Insight (Outliers & Distributions)

Several numerical features show a high number of IQR-based outliers, particularly any_medical_conditions, age_binary, probable, and travel_outside_canada.
These variables are binary in nature, so the detected “outliers” reflect class imbalance rather than true extreme values.
This indicates skewed distributions where one class dominates, suggesting the need for proportion-based analysis instead of traditional outlier removal.

In [ ]:
Why this question matters

Outliers may represent:

-Data entry errors 

-Rare but important events 

-Structural inequality in the data

-Ignoring them can invalidate statistical assumptions.

 Technical Insight

We use:

IQR (robust to skewness)

Distribution shape

In [ ]:
Question 3: Which categorical variables dominate the dataset, and do they introduce class imbalance?

In [13]:
cat_cols = df1.select_dtypes(include="object").columns

for col in cat_cols:
    print(f"\n{col}")
    print(df1[col].value_counts(normalize=True).head())


month
month
April    1.0
Name: proportion, dtype: float64

fsa
fsa
M5V    0.017083
M2N    0.010772
M5A    0.010580
M6H    0.010366
M6P    0.010094
Name: proportion, dtype: float64

age_1
age_1
<65    0.858114
>65    0.141886
Name: proportion, dtype: float64


Insight (Categorical Dominance & Class Imbalance)

The month variable is fully dominated by a single category (April = 100%), making it non-informative for analysis.
The fsa variable shows very high category fragmentation, with no single region dominating, which can lead to sparse and unstable geographic insights.
The age_1 variable exhibits class imbalance, with individuals under 65 representing ~86% of the data, potentially biasing results toward younger populations.

Why this question matters

Class imbalance:

Skews accuracy metrics

Leads to biased models

Hides minority patterns

 Technical Insight

We assess:

Frequency dominance

Long-tail categories

Question 4:Which variables have the strongest relationships, and are they redundant?

In [14]:
corr = df1[num_cols].corr()

high_corr = (
    corr.abs()
    .unstack()
    .sort_values(ascending=False)
)

high_corr = high_corr[high_corr < 1].drop_duplicates().head(10)
high_corr

any_medical_conditions  vulnerable              0.760489
probable                contact_with_illness    0.723527
vulnerable              age_binary              0.683764
shortness_of_breath     probable                0.670588
cough                   probable                0.351126
probable                fever_chills_shakes     0.347290
fever_chills_shakes     cough                   0.313543
shortness_of_breath     cough                   0.309898
fever_chills_shakes     shortness_of_breath     0.280313
any_medical_conditions  age_binary              0.201029
dtype: float64

In [ ]:
Insight
Strong correlations exist between any_medical_conditions & vulnerable (0.76), and vulnerable & age_binary (0.68), showing vulnerability is driven by age and health status.

probable is moderately correlated with contact_with_illness (0.72), indicating exposure is a key factor.

Symptoms (cough, shortness_of_breath, fever_chills_shakes) show moderate correlation, suggesting symptom clustering.

No correlations exceed 0.85, indicating low redundancy and minimal multicollinearity.

Why this question matters

Highly correlated variables:

Inflate model variance (multicollinearity)

Reduce interpretability

Waste features

Technical Insight

Correlation helps decide:

Feature elimination

Feature engineering

Dimensionality reduction

Question:5 How does the target variable behave across key groups?


Question 5 How does vulnerability vary across age groups?

In [17]:
target = "vulnerable"
group_col = "age_binary"

df1.groupby(group_col)[target].agg(
    count="count",
    mean="mean",
    median="median",
    std="std"
).sort_values(by="mean", ascending=False)

,count,mean,median,std
age_binary,,,,
1,33903,1.000000,1.0,0.000000
0,205042,0.139113,0.0,0.346065


In [ ]:
Vulnerability Across Age Groups

Individuals in age_binary = 1 show 100% vulnerability, indicating significantly higher risk.

In contrast, age_binary = 0 has a much lower vulnerability rate (~14%).

This highlights age as a critical driver of vulnerability.

Question 6 Impact of symptoms on probable cases

In [18]:
symptoms = [
    "fever_chills_shakes",
    "cough",
    "shortness_of_breath"
]

df1.groupby(symptoms)["probable"].mean().reset_index()

,fever_chills_shakes,cough,shortness_of_breath,probable
0,0,0,0,0.019849
1,0,0,1,1.000000
2,0,1,0,0.127466
3,0,1,1,1.000000
4,1,0,0,0.144006
5,1,0,1,1.000000
6,1,1,0,1.000000
7,1,1,1,1.000000


In [ ]:
Insight
Impact of Symptoms on Probable Cases

Individuals with shortness of breath show a 100% probability of being classified as probable cases.

The probability increases significantly as multiple symptoms co-occur.

Absence of symptoms is associated with a very low probability (~2%).

In [16]:
df1.columns

Index(['month', 'fsa', 'probable', 'vulnerable', 'is_most_recent',
       'fever_chills_shakes', 'cough', 'shortness_of_breath',
       'any_medical_conditions', 'travel_outside_canada',
       'contact_with_illness', 'age_1', 'age_binary'],
      dtype='object')